In [ ]:
import pandas as pd
import json
import re

In [ ]:
# Reading in OneMap Building JSON
# Source : https://github.com/xkjyeah/singapore-postal-codes/blob/master/buildings.json
building_df = pd.read_json (r"Data\buildings.json")
building_df["blk_street"] = building_df["BLK_NO"] + " " + building_df["ROAD_NAME"]

building_df.drop(['LONGTITUDE', 'X', 'Y', 'SEARCHVAL'], axis=1, inplace=True)

building_df.info()

In [ ]:
# Read in HDB Property Information 
# Source: https://data.gov.sg/dataset/hdb-property-information

hdb_df = pd.read_csv (r"Data\hdb-property-information.csv")
hdb_df['street'] = hdb_df['street'].replace(regex=[' RD'], value=' ROAD')
hdb_df['street'] = hdb_df['street'].replace(regex=[' STH'], value=' SOUTH')
hdb_df['street'] = hdb_df['street'].replace(regex=[' NTH'], value=' NORTH')
hdb_df['street'] = hdb_df['street'].replace(regex=['NTH '], value='NORTH ')
hdb_df['street'] = hdb_df['street'].replace(regex=['ST. '], value='SAINT ')
hdb_df['street'] = hdb_df['street'].replace(regex=[' CTRL'], value=' CENTRAL')
hdb_df['street'] = hdb_df['street'].replace(regex=['BT '], value='BUKIT ')
hdb_df['street'] = hdb_df['street'].replace(regex=['LOR '], value='LORONG ')
hdb_df['street'] = hdb_df['street'].replace(regex=[' AVE'], value=' AVENUE')
hdb_df['street'] = hdb_df['street'].replace(regex=[' ST'], value=' STREET')
hdb_df['street'] = hdb_df['street'].replace(regex=[' DR'], value=' DRIVE')
hdb_df['street'] = hdb_df['street'].replace(regex=['UPP '], value='UPPER ')
hdb_df['street'] = hdb_df['street'].replace(regex=[' CRES'], value=' CRESCENT')
hdb_df['street'] = hdb_df['street'].replace(regex=[' TER'], value=' TERRACE')
hdb_df['street'] = hdb_df['street'].replace(regex=['JLN '], value='JALAN ')
hdb_df['street'] = hdb_df['street'].replace(regex=[' CL'], value=' CLOSE')
hdb_df['street'] = hdb_df['street'].replace(regex=['KG '], value='KAMPONG ')
hdb_df['street'] = hdb_df['street'].replace(regex=[' GDNS'], value=' GARDENS')
hdb_df['street'] = hdb_df['street'].replace(regex=[' HTS'], value=' HEIGHTS')
hdb_df['street'] = hdb_df['street'].replace(regex=["C'WEALTH "], value='COMMONWEALTH ')
hdb_df['street'] = hdb_df['street'].replace(regex=[" PK"], value=' PARK')
hdb_df['street'] = hdb_df['street'].replace(regex=[" PL"], value=' PLACE')
hdb_df['street'] = hdb_df['street'].replace(regex=[" PLACEAINS"], value=' PLAINS')
hdb_df['street'] = hdb_df['street'].replace(regex=[" PLACEAZA"], value=' PLAZA')
hdb_df['street'] = hdb_df['street'].replace(regex=[" IND EST"], value=' INDUSTRIAL ESTATE')

hdb_df["blk_street"] = hdb_df["blk_no"] + " " + hdb_df["street"]

hdb_df = hdb_df.add_prefix('HDB_')
hdb_df = hdb_df.rename(columns={'HDB_blk_street': 'blk_street'})

hdb_df.info()

In [ ]:
merged_df = pd.merge(building_df, hdb_df, how='left', on="blk_street")

merged_df.drop(['blk_street', 'HDB_blk_no', 'HDB_street'], axis=1, inplace=True)

In [ ]:
merged_df.info()

In [ ]:
commercial_df = pd.read_excel (r"Data\Commercial Property Info.xlsx")

commercial_df = commercial_df.add_prefix('PRI_')
commercial_df = commercial_df.rename(columns={'PRI_Postal Code': 'POSTAL'})
commercial_df.drop(['PRI_Unnamed: 0', 'PRI_District Code'], inplace=True, axis=1)

merged_df = pd.merge(merged_df, commercial_df, how='left', on="POSTAL")

In [ ]:
district_df = pd.read_excel (r"Data\PostalDistrict.xlsx")

dict_district = {}
for index, row in district_df.iterrows():
    res = str(row['Postal Sector']).split(',')
    
    for r in res:
        dict_district[str(r).strip()] = row['Postal District']


In [ ]:
def computePostalDistrict(row):
    try:
        code = row['POSTAL'][:2]

        return dict_district[code]
    except:
        return ''
    

In [ ]:
merged_df['POSTAL_DISTRICT'] = merged_df.apply(lambda row: computePostalDistrict(row), axis=1)

In [ ]:
merged_df.info()

In [ ]:
writer = pd.ExcelWriter('SingaporeBuilding.xlsx')
merged_df.to_excel(writer, "address") 
writer.save()